# The Data

## Basic imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Connecting to the database

Setting up a connection to the database file is straightforward via `sqlite`. Just update the location of the data file and you're connected!

In [2]:
f_data = "email_extractor.sqlite"

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect(f_data)

The important parts of the database look like:

![database structure](./images/email_extractor_db_diagram.png "Database Structure")

There are other tables as well, but they're all around admin/permissions so shouldn't be relevant.

## Reading from the database

You can use `pandas` to read raw SQL queries using the connection that we have just made. For example, if we want to look at all of the tables in the database:

In [5]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con)

,type,name,tbl_name,rootpage,sql
0,table,accounts_user,accounts_user,2,CREATE TABLE accounts_user (\n id bigint NO...
1,table,accounts_user_groups,accounts_user_groups,3,CREATE TABLE accounts_user_groups (\n id bi...
2,table,accounts_user_user_permissions,accounts_user_user_permissions,4,CREATE TABLE accounts_user_user_permissions (\...
3,table,auth_group,auth_group,5,CREATE TABLE auth_group (\n id integer NOT ...
4,table,auth_group_permissions,auth_group_permissions,6,CREATE TABLE auth_group_permissions (\n id ...
5,table,auth_permission,auth_permission,7,CREATE TABLE auth_permission (\n id integer...
6,table,django_admin_log,django_admin_log,8,CREATE TABLE django_admin_log (\n id intege...
7,table,django_celery_beat_clockedschedule,django_celery_beat_clockedschedule,9,CREATE TABLE django_celery_beat_clockedschedul...
8,table,django_celery_beat_crontabschedule,django_celery_beat_crontabschedule,10,CREATE TABLE django_celery_beat_crontabschedul...
9,table,django_celery_beat_intervalschedule,django_celery_beat_intervalschedule,11,CREATE TABLE django_celery_beat_intervalschedu...


## Accessing emails

The emails themselves are in the `emails_email` table. There's lots of fields, but the key ones are:

- `id`: unique integer identifier for the email
- `internal_date`: the date of the email (`created_at` and `updated_at` are dates relating to the database records)
- `from_address`: the sender of the email
- `html_body`: the email body in html format (see below)
- `plain_text_body`: the email body in plain text format (see below)
- `subject`: the subject of the email
- `to`: the set of addresses that the email was sent to
- `eml_file`: the location of the email file

In [6]:
df = pd.read_sql_query("SELECT * FROM emails_email LIMIT 50000", con)

In [7]:
df.head()

,id,created_at,updated_at,uid,flags,internal_date,bcc,cc,from_address,html_body,message_id,plain_text_body,reply_to,subject,to,eml_file,folder_id
0,306340,2022-07-29 20:00:12.363819+00,2022-07-29 20:00:12.363835+00,8464.0,\Recent,2022-07-28 22:31:54+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282230.26SMUf7A003811@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/bbbf78a5-ccf2-44dd-af99-2...,3
1,306341,2022-07-29 20:00:12.927732+00,2022-07-29 20:00:12.927749+00,8465.0,\Recent,2022-07-28 22:32:29+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMV1Le003938@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/cb06c705-4beb-44d6-86db-d...,3
2,306342,2022-07-29 20:00:13.208991+00,2022-07-29 20:00:13.209005+00,8466.0,\Recent,2022-07-28 22:33:27+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMVa9Q004201@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/a9c0843e-e907-4013-b742-d...,3
3,306343,2022-07-29 20:00:13.423754+00,2022-07-29 20:00:13.423769+00,8467.0,\Recent,2022-07-28 22:37:12+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMV7cM004046@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/8216b595-d169-4521-89ee-3...,3
4,306344,2022-07-29 20:00:13.606982+00,2022-07-29 20:00:13.606995+00,8468.0,\Recent,2022-07-29 09:41:28+00,{},{},info@phoenixsafe.co.uk,"<!doctype html><html xmlns=""http://www.w3.org/...",<c2e1bcff24eed0dbef1ffd4a0.910c7a73bb.20220729...,Attention: This email originated outside of ou...,{info@phoenixsafe.co.uk},Keep Cash Safe,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/f467154d-fbb3-44cd-96cd-0...,3


## Email bodies

Email bodies can be stored in `html` format, `plain_text` format or both.

In [8]:
w_html = df.html_body.notnull() 
w_plain = df.plain_text_body.notnull()

In [9]:
w_html.sum()

38931

In [10]:
w_plain.sum()

17462

In [11]:
(w_html & w_plain).sum()

6399

Below is a method that we have used previously to convert html format bodies to text. The package `html2text` could also be used for this it seems.

In [17]:
from bs4 import BeautifulSoup, Comment

def text_from_html(body) -> str:
    if pd.isnull(body):
        return ""

    def tag_visible(element):
        if element.parent.name in ["style", "script", "head", "title", "meta", "[document]"]:
            return False
        if isinstance(element, Comment):
            return False
        return True

    soup = BeautifulSoup(body, "html.parser")
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)
    return " ".join(t.strip() for t in visible_texts)

# import html2text
# html2text.html2text(text)

In [18]:
# def text_from_html(body: str | None) -> str:
#     if pd.isnull(body):
#         return ""

In [19]:
df["html_body_as_text"] = df["html_body"].map(text_from_html)

In [103]:
df["body"] = df["plain_text_body"]

In [104]:
df.loc[df["body"].isnull(), "body"] = df.loc[df["body"].isnull(), "html_body_as_text"]

In [105]:
df["body"].isnull().value_counts()

False    50000
Name: body, dtype: int64

## Accessing email categorisations

The email categories are stored in `emails_category`. This then links to the emails themselves via the `emails_emailcategory` table.

Here, data from the three tables is pulled separately, but you can of course join the data in an SQL query.

In [106]:
df_email_categorisations = pd.read_sql_query("SELECT * FROM emails_emailcategory", con)

In [107]:
df_email_categorisations.head()

,id,created_at,updated_at,category_id,created_by_id,email_id
0,12264,2022-07-26 09:30:40.505263+00,2022-07-26 09:30:40.505285+00,101,NaN,284813
1,12265,2022-07-26 09:30:40.505309+00,2022-07-26 09:30:40.505314+00,101,NaN,284821
2,12266,2022-07-26 09:30:40.505325+00,2022-07-26 09:30:40.505329+00,101,NaN,285029
3,12267,2022-07-26 09:30:40.50534+00,2022-07-26 09:30:40.505344+00,101,NaN,285515
4,12268,2022-07-26 09:30:40.505355+00,2022-07-26 09:30:40.505359+00,101,NaN,285679


In [121]:
df_email_categories = pd.read_sql_query("SELECT * FROM emails_category", con)

One thing to note: for some reason the email categories table has duplicated entries:

In [109]:
df_email_categories.sort_values(by="name").head()

,id,created_at,updated_at,name,slug,hidden
45,34,2022-07-25 11:13:26.881888+00,2022-07-26 14:55:38.134577+00,Backorder,backorder,t
21,34,2022-07-25 11:13:26.881888+00,2022-07-26 14:55:38.134577+00,Backorder,backorder,t
1,35,2022-07-25 11:14:09.036276+00,2022-07-25 11:14:09.036299+00,Bank Detail Change,bank-detail-change,f
25,35,2022-07-25 11:14:09.036276+00,2022-07-25 11:14:09.036299+00,Bank Detail Change,bank-detail-change,f
43,36,2022-07-25 11:14:38.218737+00,2022-07-26 09:58:01.408127+00,Credit Note,credit-note,t


For now let's convert the categories table into a dictionary and map that to the categorisations table.

In [110]:
category_slug_lookup = df_email_categories.drop_duplicates(keep="first", subset=["id", "name"]).set_index("id")["slug"].to_dict()
df_email_categorisations["category_slug"] = df_email_categorisations["category_id"].map(category_slug_lookup)

In [111]:
df_email_categorisations.head()

,id,created_at,updated_at,category_id,created_by_id,email_id,category_slug
0,12264,2022-07-26 09:30:40.505263+00,2022-07-26 09:30:40.505285+00,101,NaN,284813,reminders
1,12265,2022-07-26 09:30:40.505309+00,2022-07-26 09:30:40.505314+00,101,NaN,284821,reminders
2,12266,2022-07-26 09:30:40.505325+00,2022-07-26 09:30:40.505329+00,101,NaN,285029,reminders
3,12267,2022-07-26 09:30:40.50534+00,2022-07-26 09:30:40.505344+00,101,NaN,285515,reminders
4,12268,2022-07-26 09:30:40.505355+00,2022-07-26 09:30:40.505359+00,101,NaN,285679,reminders


We can then pull out the change-of-bank-details emails, and look at a few examples. Note, there's not many examples in our dataframe because:
1. Only ~1% of emails have been categorised
2. We only pulled ~3% of emails to start with

In [112]:
df_bdc = df_email_categorisations[df_email_categorisations["category_slug"] == "bank-detail-change"]
df_bdc.head()

,id,created_at,updated_at,category_id,created_by_id,email_id,category_slug
679,906,2022-07-25 11:14:12.838098+00,2022-07-25 11:14:12.838116+00,35,NaN,282436,bank-detail-change
680,907,2022-07-25 11:14:12.838137+00,2022-07-25 11:14:12.838142+00,35,NaN,283345,bank-detail-change
681,908,2022-07-25 11:14:12.838153+00,2022-07-25 11:14:12.838157+00,35,NaN,290334,bank-detail-change
682,909,2022-07-25 11:14:12.838168+00,2022-07-25 11:14:12.838172+00,35,NaN,290391,bank-detail-change
683,910,2022-07-25 11:14:12.838183+00,2022-07-25 11:14:12.838187+00,35,NaN,291518,bank-detail-change


In [113]:
df[df.id.isin(df_bdc["email_id"])]

,id,created_at,updated_at,uid,flags,internal_date,bcc,cc,from_address,html_body,message_id,plain_text_body,reply_to,subject,to,eml_file,folder_id,html_body_as_text,body
8535,186977,2022-07-20 17:10:52.857914+00,2022-08-19 08:59:21.724909+00,390141.0,\Seen,2021-09-14 10:07:55+00,{},"{Purchase.Ledger@evo-group.co.uk,Alex.Mackay@b...",mina.pasaloglou@evo-group.co.uk,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",<LO4P265MB3613B656540D7DB92FEAFA90F2DA9@LO4P26...,None,{},RE: Tyronex Bank Account Change,"{Matt@tyronex.co.uk,Matt.Hales@evo-group.co.uk...",email_data/Evo Group/d2216e54-60dc-45b6-8120-8...,6,"Hi Matt, Thank you for your email. Can ...","Hi Matt, Thank you for your email. Can ..."
25926,186977,2022-07-20 17:10:52.857914+00,2022-08-19 08:59:21.724909+00,390141.0,\Seen,2021-09-14 10:07:55+00,{},"{Purchase.Ledger@evo-group.co.uk,Alex.Mackay@b...",mina.pasaloglou@evo-group.co.uk,"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",<LO4P265MB3613B656540D7DB92FEAFA90F2DA9@LO4P26...,None,{},RE: Tyronex Bank Account Change,"{Matt@tyronex.co.uk,Matt.Hales@evo-group.co.uk...",email_data/Evo Group/d2216e54-60dc-45b6-8120-8...,6,"Hi Matt, Thank you for your email. Can ...","Hi Matt, Thank you for your email. Can ..."


In [114]:
for row in df[df.id.isin(df_bdc["email_id"])].head(10).itertuples():
    print("#" * 80, "\n\n")
    print("SUBJECT:\n",row.subject)
    print("")
    print("BODY:\n",row.body)
    print("\n")

################################################################################ 


SUBJECT:
 RE: Tyronex Bank Account Change 

BODY:
     Hi Matt,    Thank you for your email. Can you also please provide one of the following:    -Void cheque  -Void payment slip  -A bank statement    Thank you,  Mina      From: Matt Tibble <Matt@tyronex.co.uk>  Sent: 13 September 2021 10:17  To: Matt Hales <Matt.Hales@evo-group.co.uk>; Robert Mitchell <Robert.Mitchell@banneruk.com>  Cc: Purchase Ledger <Purchase.Ledger@evo-group.co.uk>; Alex Mackay <Alex.Mackay@banneruk.com>; Mina Pasaloglou <mina.pasaloglou@evo-group.co.uk>; Accounts <accounts@tyronex.co.uk>  Subject: FW: Tyronex Bank Account Change      Alert: There is currently a serious threat from opening email attachments. If you have any concerns about an attached file please contact IT.  Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.   Hi All    Apologies, pleas

Doing a quick keyword search for "bank details" shows that most emails containing this phrase aren't *change* of bank details emails.

In [120]:
for row in df[df.plain_text_body.str.contains("bank details") == True].head(10).itertuples():
    print("#" * 80, "\n\n")
    print(row.subject)
    print("")
    print(row.body)
    print("\n")

################################################################################ 


RE: **URGENT** Invoice for services - attached (ref. 25443)
 (#0773545)

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.

-#-#- Please reply above this line -#-#-

________________________________
Chris Ridgeon
29/07/2022 11:37 AM


Good afternoon,



This is now urgent.



Please can we chase the attached invoice for payment, it is overdue and was issued in May.



Thanks,

Chris Ridgeon
Ridgeon Network Support
Office: 02476 382031

________________________________
Chris Ridgeon
19/07/2022 11:00 AM


Good morning,



Please can we have a payment date for the attached overdue invoice?



Thanks,

Chris Ridgeon
Ridgeon Network Support
Office: 02476 382031

________________________________
accounts@ridgeon-network.co.uk
31/05/2022 11:46 AM


[Ridgeon Network Ltd]<http://www.ridgeon-network.co.uk/>

Ridgeon Network Ltd – Acc